In [1]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 9.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
!pip install torchtext

In [3]:
!pip install spacy

In [4]:
def generateBigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [5]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  preprocessing = generateBigrams)

LABEL = data.LabelField(dtype = torch.float)

In [6]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(183))

aclImdb_v1.tar.gz:   1%|          | 803k/84.1M [00:00<00:11, 7.55MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 41.1MB/s]


In [8]:
TEXT.build_vocab(train_data, 
                 max_size = 30_000, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

100%|█████████▉| 398601/400000 [00:30<00:00, 26522.37it/s]

In [9]:
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = batch_size, 
    device = device)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
                
        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        return self.fc(pooled)

In [11]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [12]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.5229,  0.2123, -0.9764,  ...,  0.8045,  0.6421, -0.7721],
        [-0.7215,  0.6692,  0.4192,  ..., -0.2213,  3.0214,  0.2254],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.3468,  0.1875,  0.4697,  ...,  1.0673, -0.3361,  0.4291],
        [ 0.8974,  0.8951,  0.1663,  ...,  1.2212, -0.8497,  0.7306],
        [ 0.3690,  2.2404, -0.6675,  ..., -1.3191, -0.9617, -0.2470]])

In [13]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [15]:
def binaryAccuracy(preds, y):
    # Returns accuracy per batch and always round predictions to closest int
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [16]:
def train_fast_text(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binaryAccuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
def evaluate_fast_text(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binaryAccuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Training model

In [19]:
epochs = 7

best_valid_loss = float('inf')

for epoch in range(epochs):

    start_time = time.time() #start time
    
    train_loss, train_acc = train_fast_text(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_fast_text(model, valid_iterator, criterion)
    
    end_time = time.time() #end time

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'ft_model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 11s
	Train Loss: 0.678 | Train Acc: 61.93%
	 Val. Loss: 0.562 |  Val. Acc: 76.31%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.581 | Train Acc: 79.04%
	 Val. Loss: 0.393 |  Val. Acc: 82.44%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.457 | Train Acc: 85.90%
	 Val. Loss: 0.349 |  Val. Acc: 85.95%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.371 | Train Acc: 88.98%
	 Val. Loss: 0.358 |  Val. Acc: 87.54%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.312 | Train Acc: 90.63%
	 Val. Loss: 0.372 |  Val. Acc: 88.00%
Epoch: 06 | Epoch Time: 0m 11s
	Train Loss: 0.272 | Train Acc: 91.81%
	 Val. Loss: 0.391 |  Val. Acc: 88.67%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.240 | Train Acc: 92.65%
	 Val. Loss: 0.412 |  Val. Acc: 88.94%


In [21]:
model.load_state_dict(torch.load('ft_model.pt'))

test_loss, test_acc = evaluate_fast_text(model, test_iterator, criterion)

print(f'Test Loss is: {test_loss:.3f} | Test Acc is: {test_acc*100:.2f}%')

Test Loss is: 0.365 | Test Acc is: 85.23%


## User Input



In [22]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generateBigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [23]:
predict_sentiment(model, "This film is not terrible")

0.9999287128448486

In [24]:
predict_sentiment(model, "This film is great")

1.5930143055542492e-18